In [1]:
import pandas as pd 
import numpy as np

import plotly.express as px

In [2]:
df = pd.read_csv("dataset_main.csv")

In [5]:
big_list = []
count = 0
for one in df['serial_number']:
  if one not in big_list:
    count += 1
    big_list.append(one)

print(f"Uniques: {count}")


Uniques: 100


In [7]:
df.head(10)

,serial_number,manufacturer,kva,age,region,failure_date,failure_cause
0,T1012273,Siemens,45.0,1,Coastal,1/17/2022,Animal
1,T1083095,Siemens,75.0,1,Mountain,8/14/2022,Fire
2,T1010763,ABB,15.0,1,Mountain,10/13/2022,Overload
3,T1070899,Schneider,75.0,2,North,9/5/2022,Fire
4,T1097583,General Electric,45.0,2,Coastal,3/20/2022,Corrosion
5,T1060963,Schneider,45.0,2,Mountain,5/5/2022,Fire
6,T1015855,ABB,30.0,3,North,8/2/2022,Car Hit Pole
7,T1065758,Schneider,15.0,3,North,11/3/2022,Corrosion
8,T1079245,Siemens,45.0,3,Desert,11/20/2022,Corrosion
9,T1072194,ABB,112.5,3,Desert,8/7/2022,Animal


In [4]:
df['manufacturer'][3]

'Schneider'

In [3]:
df['Dummy'] = [1 for _ in range(100)]

In [12]:
df.head()

,Transformer Serial No.,Mfg Code,Manufacturer,KvA Code,KvA,Age,Reg Code,Region,Failure Date,Cause Code,Failure Cause,Dummy
0,T1012273,2,Siemens,3,45.0,1,2,Coastal,1/17/2022,3,Animal,1
1,T1083095,2,Siemens,4,75.0,1,4,Mountain,8/14/2022,4,Fire,1
2,T1010763,4,ABB,1,15.0,1,4,Mountain,10/13/2022,1,Overload,1
3,T1070899,3,Schneider,4,75.0,2,1,North,9/5/2022,4,Fire,1
4,T1097583,1,General Electric,3,45.0,2,2,Coastal,3/20/2022,5,Corrosion,1


In [5]:
fig = px.bar(df, x="age", y="Dummy", color='manufacturer', barmode='group')
fig.show()

In [16]:
fig = px.bar(df, x="", y="Dummy", color="Failure Cause", barmode='group')
fig.show()